In [1]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()


data = pd.read_csv("data/news.csv")
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


## Data Preprocessing

In [2]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])


In [4]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1


### Tokenization

In [5]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])


In [ ]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1,  padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

### Generating Word Embedding

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

# Step 1: Load Pre-trained GloVe Embeddings
print("Loading GloVe embeddings...")

# Make sure 'glove.6B.50d.txt' is in your working directory
glove_file = "glove.6B.50d.txt"

embeddings_index = {}

# Open file with UTF-8 encoding to avoid errors
with open(glove_file, "r", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # First word in the line is the word itself
        coefs = np.asarray(values[1:], dtype="float32")  # Rest are embedding values
        embeddings_index[word] = coefs

print(f"Loaded {len(embeddings_index)} word vectors.")

# Step 2: Tokenize Sample Text
texts = ["This is a sample text", "GloVe embeddings are useful for NLP"]  # Example corpus

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

word_index1 = tokenizer.word_index  # Dictionary mapping words to indices
vocab_size1 = len(word_index1)  # Vocabulary size
embedding_dim = 50  # Using 50-dimensional GloVe vectors

print(f"Vocabulary Size: {vocab_size1}")

# Step 3: Generate Embedding Matrix
embeddings_matrix = np.zeros((vocab_size1 + 1, embedding_dim))  # Initialize matrix

for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)  # Get GloVe vector for word
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector  # Assign vector to matrix

print("Embedding matrix successfully created!")


Loading GloVe embeddings...
Loaded 400000 word vectors.
Vocabulary Size: 11
Embedding matrix successfully created!
